
# Simple Stocks Data API — Colab Quickstart

This notebook fetches **AAPL** data from the API on RapidAPI and:
- Loads **price history** and plots the last 180 daily closes
- Retrieves **financial statements**
- Retrieves **earnings estimates**

**Before you run:** set your RapidAPI key below.


In [ ]:
# Paste your RapidAPI key
RAPIDAPI_KEY = "YOUR_RAPIDAPI_KEY"  # <-- replace this
BASE = "https://macrotrends-finance1.p.rapidapi.com"
HOST = "macrotrends-finance1.p.rapidapi.com"
SYMBOL = "AAPL"

assert RAPIDAPI_KEY and RAPIDAPI_KEY != "YOUR_RAPIDAPI_KEY", "Please set RAPIDAPI_KEY first."


In [ ]:
# Install dependencies in Colab if needed
import sys
if "google.colab" in sys.modules:
    %pip -q install requests pandas matplotlib > /dev/null


In [ ]:
import requests, pandas as pd, pprint

H = {"X-RapidAPI-Key": RAPIDAPI_KEY, "X-RapidAPI-Host": HOST}

# --- Price History ---
r_prices = requests.get(f"{BASE}/price-history/{SYMBOL}", headers=H, timeout=30)
r_prices.raise_for_status()
prices = r_prices.json()
df = pd.DataFrame.from_dict(prices, orient="index").sort_index()
display(df.tail())

# --- Financial Statements ---
r_fin = requests.get(f"{BASE}/financial-statements/{SYMBOL}", headers=H, timeout=30)
r_fin.raise_for_status()
fin = r_fin.json()
print("Financial statements keys:", list(fin.keys())[:10])

# --- Earnings Estimates ---
r_est = requests.get(f"{BASE}/earnings-estimates/{SYMBOL}", headers=H, timeout=30)
r_est.raise_for_status()
est = r_est.json()
print("Earnings estimates top-level keys:", list(est.keys()))
pprint.pp({k: est.get(k) for k in list(est.keys())[:2]})


In [ ]:
# Plot last 180 closes using matplotlib (no seaborn)
import matplotlib.pyplot as plt

# Be tolerant if column name is 'close' or 'Close'
close_col = 'close' if 'close' in df.columns else ('Close' if 'Close' in df.columns else None)
if close_col is None:
    raise ValueError("Could not find 'close' or 'Close' column in the response. Inspect df.columns")

last = df.tail(180)
plt.figure(figsize=(10,4))
plt.plot(last.index, last[close_col])
plt.title(f"{SYMBOL} — Last 180 daily closes")
plt.xlabel("Date")
plt.ylabel("Close")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
